# Different Subjects

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import os

from torch.utils.data import DataLoader
from data.image_dataset import ImageDataset
from torchvision.transforms import v2
from torchvision.models import inception_v3
from models.convnet import ConvNet
from models.nvgaze import NVGaze
from models.incep_convnet import IncepConvNet
from training.train import *
from training.eval import *
from data.convert_labels import *
from utils.dataset_utils import check_disjoint
import matplotlib.pyplot as plt
from datetime import datetime

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [3]:
for subject in [1, 2, 3, 4, 5, 13, 14, 15, 16, 17]:
  stride = 20
  input_dims = (1, 120, 160) # (C, H, W)
  note = ''
  mode = 'reg'
  batch_size = 64

  data_subset = f'{subject:>02}_{stride:>02}_{input_dims[2]}_{input_dims[1]}{f'_{note}' if note else ''}'
  train_dir_path = f'data/real/{data_subset}/train'
  train_annotations_file_path = f'{train_dir_path}/{data_subset}_train.csv'
  val_dir_path = f'data/real/{data_subset}/val'
  val_annotations_file_path = f'{val_dir_path}/{data_subset}_val.csv'
  test_dir_path = f'data/real/{data_subset}/test'
  test_annotations_file_path = f'{test_dir_path}/{data_subset}_test.csv'

  target_transform = None

  train_dataset = ImageDataset(
    annotations_file=train_annotations_file_path,
    img_dir=train_dir_path,
    transform=None,
    target_transform=target_transform
  )
  val_dataset = ImageDataset(
    annotations_file=val_annotations_file_path,
    img_dir=val_dir_path,
    transform=None,
    target_transform=target_transform
  )
  test_dataset = ImageDataset(
    annotations_file=test_annotations_file_path,
    img_dir=test_dir_path,
    transform=None,
    target_transform=target_transform
  )

  # assert(check_disjoint(val_dataset, test_dataset))

  # Create Dataloader
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

  out_num_features = 2

  model = NVGaze(input_dims, out_num_features, dropout_param=0.05)

  criterion = nn.MSELoss() if mode == 'reg' else nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=1e-3)
  scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50, 180], gamma=0.01)

  # Set-up GPU device (if available) and move model
  device = torch.device('mps' if torch.mps.is_available else 'cpu')
  model = model.to(device)
  print(f'Using device: {device}')

  results = train(model, device, mode, criterion, optimizer, train_loader, val_loader, epochs=200)

  train_error = evaluate_error(model, device, train_loader)
  val_error = evaluate_error(model, device, val_loader)
  test_error = evaluate_error(model, device, test_loader)
  train_error_str = f'Train err: {train_error.item():.6f} ({np.rad2deg(train_error.item()):.2f}°)'
  val_error_str = f'Val err:   {val_error.item():.6f} ({np.rad2deg(val_error.item()):.2f}°)'
  test_error_str = f'Test err:  {test_error.item():.6f} ({np.rad2deg(test_error.item()):.2f}°)'
  error_summary = [train_error_str, val_error_str, test_error_str]
  error_summary_str = '\n'.join(error_summary)
  print(f'{subject:>02}')
  print(error_summary_str)
  print()

Using device: mps
01
Train err: 0.038884 (2.23°)
Val err:   0.050884 (2.92°)
Test err:  0.109462 (6.27°)

Using device: mps
02
Train err: 0.041705 (2.39°)
Val err:   0.097623 (5.59°)
Test err:  0.060877 (3.49°)

Using device: mps
03
Train err: 0.048570 (2.78°)
Val err:   0.117798 (6.75°)
Test err:  0.162527 (9.31°)

Using device: mps
04
Train err: 0.034258 (1.96°)
Val err:   0.075707 (4.34°)
Test err:  0.057463 (3.29°)

Using device: mps
05
Train err: 0.036805 (2.11°)
Val err:   0.042696 (2.45°)
Test err:  0.047802 (2.74°)

Using device: mps
13
Train err: 0.031689 (1.82°)
Val err:   0.060480 (3.47°)
Test err:  0.042604 (2.44°)

Using device: mps
14
Train err: 0.035074 (2.01°)
Val err:   0.055475 (3.18°)
Test err:  0.057814 (3.31°)

Using device: mps
15
Train err: 0.030860 (1.77°)
Val err:   0.064477 (3.69°)
Test err:  0.062886 (3.60°)

Using device: mps
16
Train err: 0.050366 (2.89°)
Val err:   0.105946 (6.07°)
Test err:  0.089211 (5.11°)

Using device: mps
17
Train err: 0.037106 (2.13